- นำเข้าไลบรารีที่ใช้

In [ ]:

# - Groq: สำหรับการเข้าถึง LLM API
# - QdrantClient: สำหรับการใช้งาน Qdrant
# - SentenceTransformer: สำหรับการสร้างเวกเตอร์จากข้อความ
# - Langchain: สำหรับการสร้าง RetreivalQA chain และการจัดการ vector store
# - HuggingFaceEmbeddings: สำหรับการสร้าง embedding จากข้อความ
# - PromptTemplate: สำหรับการสร้าง template ของคำถาม

import os
from groq import Groq
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

- ตั้งค่า API Key

In [7]:
# 1. โหลด environment variables จากไฟล์ .env
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

- เริ่มต้น Qdrant (แบบ In-Memory)

In [8]:
#เริ่มต้น Qdrant (แบบ In-Memory)
qdrant_client = QdrantClient(":memory:")  # ใช้ In-Memory

In [9]:
# สร้าง Collection สำหรับเก็บเวกเตอร์
qdrant_client.recreate_collection(
    collection_name="documents",

    # ใช้ 384-D embedding
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

C:\Users\jeera\AppData\Local\Temp\ipykernel_40716\458410475.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

- เตรียมข้อมูลเอกสาร

In [10]:
# เตรียมข้อมูลเอกสาร

documents = [
    "อุทยานแห่งชาติดอยภูคา ที่เที่ยวน่าน อุทยานแห่งชาติดอยภูคา จ.น่าน อุทยานที่เต็มไปด้วยความอุดมสมบูรณ์ทางธรรมชาติและมีอากาศที่บริสุทธิ์ตลอดทั้งปี และที่เป็นไฮไลท์ของดอยภูคาเลยก็คือ ดอกชมพูภูคา ที่จะบานช่วงเดือนกุมภาพันธ์ และยังต้นไม้หายากที่มีที่นี่ที่เดียวในเมืองไทย นอกจากนี้ทางอุทยานฯ ยังมีบ้านพักและพื้นที่สำหรับกางเต็นท์ไว้รองรับนักท่องเที่ยว หน้าหนาวที่จะถึงนี้ ลองหนีมาเที่ยว มาสูดโอโซนให้เต็มปอดและมานอนที่ดอยภูคากันสักคืน รับรองความฟินที่ต้องมาสัมผัสให้ได้สักครั้ง",
    
    "1715 อุทยานแห่งชาติดอยภูคา ที่เที่ยวน่าน จุดชมวิว 1715 เป็นจุดชมวิวระหว่างทางลงไปเมืองปัวเพื่อกลับเข้าเมืองเป็นละทางกับที่เราขึ้นเขาทางโค้งไม่เยอะมากเหมือนตอนขาขึ้นและระยะทางลงเขายังสั้นกว่ามากคนในพื้นที่แนะนำให้ใช้เส้นทางนี้ จุดชมวิว 1715 สวยงามมากขนาดเรามาถึงใกล้จะมึดแล้วเขาสูงสลับกันไปมาเริ่มมีหมอกบางๆ นิดๆ อากาศเริ่มเย็นจัด",

    "ดอยภูแว : ดอยภูแว ที่เที่ยวน่านธรรมชาติเป็นยอดเขาสูง 1837 เมตรจากระดับน้ำทะเล ในพื้นที่ของอุทยานแห่งชาติดอยภูคา จังหวัดน่านโดยทางเดินช่วงแรกเดินขึ้นจากหน่วย ฯ ระยะทางสั้น ๆ แต่ชันพอสมควร ไปเจอถนนลูกรังไปยังหมู่บ้านปูดู่ 2 กม. เดินผ่านไปท้ายหมู่บ้านเลี้ยวซ้ายขึ้นไปผ่านไร่ข้าวโพด เดินสบาย ๆ ชมวิวเขาทั้งลูกมีแต่ต้นข้าวโพดอีก 2 กม. ผ่านรั้วเข้าเขตป่า จุดนี้ชันสุดๆ ตามต้นไม้มีจะรอยบากบอกทางเป็นระยะ ๆ อีก 2 กม. ถึงลานโล่ง ๆ เป็นจุดกางเต็นท์ มองขึ้นไปเห็นยอดดอย ส่วนตัวใช้เวลา 3 ชั่วโมงทั่วไปใช้เวลาเฉลี่ย 4-5 ชม. ด้านบนไม่มีสิ่งอำนวยความสะดวกใดๆ ไม่มีห้องน้ำ ไม่มีสัญญาณ",

    "อาหารเช้า&เฉาก๊วยนมสด :วัดพระธาตุเขาน้อย ที่เที่ยวน่านในเมืองโดยตั้งอยู่บนยอด ดอยเขาน้อย ทางด้านตะวันตกของน่าน องค์พระธาตุตั้งอยู่บนยอดดอยเขาน้อย สูงจากระดับน้ำทะเลประมาณ 240 เมตรเลยทีเดียว เป็นเจดีย์ก่ออิฐถือปูนทั้งองค์ งดงามแบบศิลปะพม่าผสมล้านนา นอกจากเราจะได้มาสักการะบูชาพระธาตุคู่บ้านคู่เมืองน่านกันแล้ว ตรงจุดลานปูนนั้น ประดิษฐานพระพุทธรูปองค์ใหญ่ คือ พระพุทธมหาอุตมมงคลนันทบุรีศรีเมืองน่าน และในบริเวณที่ประดิษฐาน พระพุทธมหาอุตมมงคลนันทบุรีศรีเมืองน่าน อีกทั้งยังเป็นจุดชมวิวทะเลหมอก ในยามเช้า ที่เราจะสามารถมองเห็นหมอกปกคลุมไปทั่วบริเวณสวยงามจริงๆ รวมไปถึงการเห็นวิวของเมืองน่านได้ทั้งเมือง โดยมีภูเขาเป็นฉากหลังที่โอบล้อมเมืองอยู่อย่างสวยงามอลังการมากทีเดียว",

    "บ่อเกลือสินเธาว์ : ที่เที่ยวน่าน บ่อเกลือสินเธาว์ นั้นเรียกได้ว่านอกจากบรรยากาศของสถานที่จะดีแล้ว เหล่าพ่อค้าแม่ค้าน่ารัก พูดจาดีกันทั้งนั้น สามารถเข้าไปถ่ายรูปได้ ที่จอดรถไม่ค่อยมี ต้องจอดตามข้างทางแล้วค่อยเดินมา ของฝากจากที่นี่ก็คงจะเป็นเกลือ เพราะมาบ่อเกลือ มีทั้งเกลือแช่เท้า เกลือสมุนไพร เกลือทำกับข้าว ฯลฯ มีให้เลือกมากมายหลายราคาตามกำลังที่จะซื้อกันเลย",

    "โรงเรียนชาวนาตำบลศิลาเพชร (Farmer School): ที่เที่ยวน่าน ฟาร์มสเตย์โรงเรียนชาวนา ที่พักใจกลางทุ่งนา ในราคาหลักร้อย แต่บรรยากาศหลักล้าน ที่พักอยู่ในอำเภอปัว จ.น่าน มีทั้งแนวห้องแอร์ และพัดลม แต่ใครที่มาพักช่วงหน้าหนาวบอกเลยว่าอากาศเย็นมาก นอกจากนั้นยังมีมุมถ่ายรูปสวยๆ หลายมุมเลย ทำให้การถ่ายรูปลงโซเชียลหรือเก็บภาพไว้ในความทรงจำ จะมีแต่ความสวยงามจนลืมไม่ลง และอยากจะกลับมาซ้ำอีกแน่นอน",

    "วัดปรางค์ : ที่เที่ยวน่าน วัดปรางค์ พลาดไม่ได้ หากมาอำเภอปัว ต้องได้ชม และสัมผัส กับ ต้นดิกเดียม ต้นไม้ที่เป็น Unseen Thailand ที่อยู่คู่วัดปรางค์แห่งอำเภอปัวมานานนับร้อยปี มากราบสักการะ พระปรางค์มุณีศรีไตรเขต และ พระธาตุบุญนาค ที่เป็นที่เคารพบูชาของผู้คนในตำบลปัว เป็นอีกหนึ่งที่เที่ยวน่านในเมืองที่สามารถได้ทั้งบุญ และบรรยากาศดี ๆ จนเกิดความสุขไม่ว่าจะทางกายหรือทางใจ",

    "วัดภูมินทร์ :ที่เที่ยวน่าน วัดภูมินทร์ เป็นวัดอารามหลวงใจกลางเมืองน่าน เป็นวัดที่มีภาพวาดในสมัยโบราณที่สวยงามตามรูปแบบของชาวล้านนา มีภาพของปู่ม่านย่าม่านกับภาพวาดกระซิบรัก ณ น่าน อันมีชื่อเสียงที่ไม่ว่าใครก็ต้องมารับชมด้วยตาของตัวเอง และยังมีพระพุทธรูปองค์ใหญ่ 4 ด้านให้เราได้กราบสักการะ โบสถ์เก่าแก่และสวยงาม เพื่อให้ได้ขอพร สำหรับความเจริญ",
]

- แปลงข้อความเป็นเวกเตอร์ และเพิ่มลงใน Qdrant

In [11]:
# แปลงข้อความเป็นเวกเตอร์ และเพิ่มลงใน Qdrant
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # โหลดโมเดลสำหรับทำ Embedding
vectors = embedding_model.encode(documents).tolist()  # แปลงข้อความเป็นเวกเตอร์

In [12]:
# เพิ่มข้อมูลลง Qdrant
points = [PointStruct(id=i, vector=vectors[i], payload={"text": documents[i]}) for i in range(len(documents))]
qdrant_client.upsert(collection_name="documents", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

- สร้างฟังก์ชันการค้นหาเอกสาร

In [13]:
# สร้างฟังก์ชันการค้นหาเอกสาร
def search_documents(query):
    query_vector = embedding_model.encode([query])[0].tolist()
    search_results = qdrant_client.search(
        collection_name="documents",
        query_vector=query_vector,
        # ดึงเอกสารที่เกี่ยวข้อง 5 อันดับแรก
        limit=5  
    )
    return [hit.payload["text"] for hit in search_results]

- System prompt

In [14]:
system_prompt = """
คุณคือ AI chatbot ผู้ช่วยแนะนำสถาน ที่ท่องเที่ยว
ให้ตอบ เฉพาะข้อมูลที่อยู่ ในเอกสารเท่านั้น
หากผู้ใช้สอบถามสถานที่ที่ไม่อยู่ในจังหวัดน่าน ให้ตอบว่า 
'ขออภัย สถานนั้นไม่มีข้อมูลอยู่
"""

- Temperature

In [15]:
# 👈 กำหนดระดับความสร้างสรรค์ (0.0 = เป๊ะ, 1.0 = สร้างสรรค์สูง)
temperature = 0.7

In [16]:
# ฟังก์ชันสร้างคำตอบ
def generate_answer(query):
    retrieved_docs = search_documents(query)
    context = "\n".join(retrieved_docs)
    full_prompt = f"{system_prompt}\n\nข้อมูลอ้างอิง:\n{context}\n\nคำถาม: {query}\n\nคำตอบ:"

    # debug 
    # print(full_prompt)

    groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))
    response = groq_client.chat.completions.create(
        # model LLM 
        model="llama-3.1-8b-instant",
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": full_prompt}
        ]
    )
    return response.choices[0].message.content

In [17]:
# 🔥 ทดสอบถามคำถาม!
query = "ขอรีวิว บ่อเกลือสินเธาว์"
answer = generate_answer(query)
print("คำถาม:", query)
print("คำตอบ:", answer)

C:\Users\jeera\AppData\Local\Temp\ipykernel_40716\1114944128.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant_client.search(


คำถาม: ขอรีวิว บ่อเกลือสินเธาว์
คำตอบ: บ่อเกลือสินเธาว์ นี่เป็นหนึ่งในสถานที่ท่องเที่ยวที่น่าสนใจในจังหวัดน่าน บ่อเกลือสินเธาว์ มีบรรยากาศที่ดี และมีพ่อค้าแม่ค้าอยู่ที่นั่น พูดจาดี และมีเกลือให้เลือกมากมายหลายราคาตามกำลังที่จะซื้อกันเลย นอกจากนี้ยังมีที่จอดรถที่ไม่ค่อยมี แต่สามารถจอดตามข้างทางได้


In [18]:
# 🔥 ทดสอบถามคำถาม!
query = "ขอรีวิว วัดภูมินทร์"
answer = generate_answer(query)
print("คำถาม:", query)
print("คำตอบ:", answer)

C:\Users\jeera\AppData\Local\Temp\ipykernel_40716\1114944128.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant_client.search(


คำถาม: ขอรีวิว วัดภูมินทร์
คำตอบ: วัดภูมินทร์ เป็นวัดอารามหลวงใจกลางเมืองน่าน เป็นวัดที่มีภาพวาดในสมัยโบราณที่สวยงามตามรูปแบบของชาวล้านนา มีภาพของปู่ม่านย่าม่านกับภาพวาดกระซิบรัก ณ น่าน อันมีชื่อเสียงที่ไม่ว่าใครก็ต้องมารับชมด้วยตาของตัวเอง และยังมีพระพุทธรูปองค์ใหญ่ 4 ด้านให้เราได้กราบสักการะ โบสถ์เก่าแก่และสวยงาม เพื่อให้ได้ขอพร สำหรับความเจริญ


In [19]:
# 🔥 ทดสอบถามคำถาม!
query = "ขอรีวิววัดใน จังหวัดน่าน มีวัดอะไรบ้าง"
answer = generate_answer(query)
print(answer)

C:\Users\jeera\AppData\Local\Temp\ipykernel_40716\1114944128.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant_client.search(


ข้อมูลที่เกี่ยวข้องกับวัดในจังหวัดน่าน

1. วัดปรางค์ - เป็นวัดที่มีต้นดิกเดียมเป็น Unseen Thailand ที่อยู่คู่วัดมานานนับร้อยปี
2. วัดภูมินทร์ - เป็นวัดอารามหลวงใจกลางเมืองน่าน มีภาพวาดในสมัยโบราณที่สวยงาม
3. วัดพระธาตุเขาน้อย - เป็นวัดที่ตั้งอยู่บนยอดดอยเขาน้อย มีพระพุทธรูปองค์ใหญ่และเจดีย์ก่ออิฐถือปูนทั้งองค์


- ตัวอย่าง Chatbot RAG

In [20]:
# ตัวอย่าง Chatbot RAG
def chatbot():
    print("สวัสดี! ฉันคือ Chatbot ที่ช่วยตอบคำถามเกี่ยวกับจังหวัดน่าน พิมพ์ 'ออก' เพื่อจบการสนทนา")
    while True:
        query = input("คุณ: ")
        if query.lower() in ["ออก", "exit", "bye"]:
            print("Bot: ลาก่อน!")
            break
        answer = generate_answer(query)
        print("Bot:", answer)

# เรียกใช้ Chatbot
chatbot()

สวัสดี! ฉันคือ Chatbot ที่ช่วยตอบคำถามเกี่ยวกับจังหวัดน่าน พิมพ์ 'ออก' เพื่อจบการสนทนา


C:\Users\jeera\AppData\Local\Temp\ipykernel_40716\1114944128.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant_client.search(


Bot: ขออภัย ฉันไม่พบคำถาม
Bot: ข้อมูลที่เกี่ยวข้องกับสถานที่ท่องเที่ยวในจังหวัดน่านมีดังนี้:

1. วัดพระธาตุเขาน้อย
2. บ่อเกลือสินเธาว์
3. วัดปรางค์
4. อุทยานแห่งชาติดอยภูคา
5. วัดภูมินทร์

หากคุณต้องการทราบข้อมูลเพิ่มเติมเกี่ยวกับสถานที่เหล่านี้หรือสอบถามสถานที่ที่น่าสนใจในจังหวัดน่าน คุณสามารถถามได้เลย!
Bot: มี 5 สถานที่ท่องเที่ยวในจังหวัดน่าน ดังนี้

1. วัดพระธาตุเขาน้อย
2. บ่อเกลือสินเธาว์
3. วัดปรางค์
4. อุทยานแห่งชาติดอยภูคา
5. วัดภูมินทร์
Bot: สวัสดีครับ! ฉันยินดีที่จะช่วยแนะนำสถานที่ท่องเที่ยวในจังหวัดน่านให้คุณครับ
Bot: ลาก่อน!
